<a href="https://colab.research.google.com/github/MarkovMarkowitz/MarkovMarkowitz/blob/main/ON5_GamblersRuin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Two Absorbing States: Gambler's Ruin**

Now, consider the following situation. A gambler bets on the outcome of a sequence of independent fair coin tosses. With each heads, the gambler gains one dollar. With each tails, the gambler loses one dollar. The gambler stops betting after reaching a fortune of $\overline{S}$ dollars or after emptying their pockets.

*   What are the probabilities of each stopping outcome?
*   How long will it take for the gambler, in expectation, to arrive at one of the stopping outcomes?

To answer these questions, we can model this setting as a Markov chain on the state space $\mathcal{S}\in\{0,1,...,\overline{s}\}$. The gambler starts with initial money $k\in\mathcal{S}$, and $s_t$ represents the money in the gambler's pocket at time $t$. Thus, we have that, for $0\lt s_t \lt \overline{s}$:

*   $\mathbb{P}(s_{t+1}=s_t+1|s_{t})=0.5$
*   $\mathbb{P}(s_{t+1}=s_t-1|s_{t})=0.5$

States 0 and $\overline{s}$ are absorbing states because any sequence of draws from the Markov chain stops after reaching any of those situations. Alternatively, we can think that $\mathbb{P}(s_{t+1}=s_t|s_{t}=\overline{s})=\mathbb{P}(s_{t+1}=s_t|s_{t}=0)=1$. We can then represent the $(\overline{s}+1)\times(\overline{s}+1)$ transition matrix as:
$$
\begin{align}
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 &\cdots & 0 \\
0.5 & 0 & 0.5 & 0 & 0 &\cdots & 0 \\
0 & 0.5 & 0 & 0.5 & 0 & \cdots & 0 \\
\vdots & \vdots & \vdots & \vdots & \vdots& \cdots & \vdots \\
0 & 0 & 0 & 0.5 & 0 & 0.5 & 0 \\
0 & 0 & 0 & 0 & 0.5 & 0 & 0.5 \\
0 & 0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}
\end{align}
$$
Before solving this with math, let's see some Monte Carlo simulation results (in this example, the gambler stops betting after reaching a fortune of 5 USD and starts with 1 USD):

In [ ]:
# LIBRARIES WE USE IN THE NOTEBOOK
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import rand, seed
from scipy.stats import norm

# seed random number generator
seed(324)

p = 0.55

TARGET_PURSE = 100
INIT_PURSE = 10

PROFIT = (TARGET_PURSE - INIT_PURSE) / INIT_PURSE * 100

N_STATES = TARGET_PURSE + 1

S = np.zeros((N_STATES, 1))
P = np.zeros((N_STATES, N_STATES))

P[0, 0] = 1.0
P[N_STATES - 1, N_STATES - 1] = 1.0

for ii in range(1, N_STATES - 1):
    for jj in range(0, N_STATES):
        if jj == ii - 1:
            P[ii, jj] = 1-p
        if jj == ii + 1:
            P[ii, jj] = p

print("Transition matrix:\n", P)

Transition matrix:
 [[1.   0.   0.   ... 0.   0.   0.  ]
 [0.45 0.   0.55 ... 0.   0.   0.  ]
 [0.   0.45 0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.55 0.  ]
 [0.   0.   0.   ... 0.45 0.   0.55]
 [0.   0.   0.   ... 0.   0.   1.  ]]


In [ ]:
N_HISTORIES = 100000  # number of histories or simulations
LEN_HIST = 100  # Length of each simulation
histories = np.zeros((N_HISTORIES, LEN_HIST))
histories[:, 0] = INIT_PURSE * np.ones(N_HISTORIES)
randarray = rand(N_HISTORIES, LEN_HIST)

for i in range(0, N_HISTORIES):
    for j in range(1, LEN_HIST):
        histories[i, j] = (
            histories[i, j - 1] + (randarray[i, j] >= 1-p) - (randarray[i, j] < 1-p)

        )
        if histories[i, j] == TARGET_PURSE or histories[i, j] < 1:
            histories[i, j + 1 : LEN_HIST + 1] = histories[i, j]  # noQA E203
            break

target_num = np.sum(np.max(histories, axis=1) == TARGET_PURSE)

end_gamble = np.zeros(N_HISTORIES)
end_gamble_sum = 0

for i in range(0, N_HISTORIES):
    if np.max(histories[i, :]) == TARGET_PURSE:
        where_gamble_ends_T = np.where((histories[i, :] == TARGET_PURSE))
        end_gamble[i] = where_gamble_ends_T[0][0]
        end_gamble_sum += 1
    elif np.min(histories[i, :]) < 1:
        where_gamble_ends_0 = np.where((histories[i, :] < 1))
        end_gamble[i] = where_gamble_ends_0[0][0]
        end_gamble_sum += 1
    else:
        end_gamble[i] = 0.0

broke_num = np.sum(np.min(histories, axis=1) < 1)

print(
    f"Chance of getting the profit of %{PROFIT}: %{100*np.round(target_num / N_HISTORIES,2)}",
    f"\nChance of losing all the money: %{100*np.round(broke_num / N_HISTORIES,2)}")
print(
    "Expected days to reach the target:",
    np.round(np.sum(end_gamble) / end_gamble_sum,1), "Trade days",

    "\nTotal number of simulations:",
    end_gamble_sum,
)

Chance of getting the profit of %900.0: %0.0 
Chance of losing all the money: %9.0
Expected days to reach the target: 49.6 Trade days 
Total number of simulations: 9091


In [ ]:

# Random walk of gamblers money to show Gambler Ruin in roulette
# Gambler tries to achieve a goal money by betting one dollar every spin in roulette
# What is the probability that the gambler is not ruined i.e. gambler doesnt run out of money?
# Run simulations of several episodes and count proportion of episodes in which the gambler is not ruined! Each episode ends when the gambler is ruined or when he achieves goal money.

PWIN = 0.491

TARGET_PURSE = 100
INIT_PURSE = 10

import numpy as np

class Gambler():
    def __init__(self):
        self.initial_money=INIT_PURSE
        self.current_money=self.initial_money
        self.goal_money=TARGET_PURSE
        self.bets_won=0
        self.trials=0

    def update_money(self,outcome):
        self.current_money+=outcome
        self.trials+=1
        if outcome==1:
            self.bets_won+=1

        if self.current_money==self.goal_money or self.current_money==0:
            result=self.current_money/self.goal_money
            return True,result,self.trials
        else:
            return False,0,self.trials


class Roulette():
    def __init__(self):
        self.win_prob= PWIN

    def play_roulette(self):
        fate=np.random.random_sample()
        if fate<self.win_prob:
            outcome=1
        else:
            outcome=-1

        return outcome


episodes=10000
finished_episodes=np.arange(int(episodes/10),episodes,int(episodes/10))

gambler = Gambler()
roulette = Roulette()

episode_wins=0
episode_spins=np.zeros(episodes)
episode_betswon=np.zeros(episodes)

for i in range(episodes):
    if i in finished_episodes:
        print('Completed {} episodes'.format(i))

    end = False
    gambler.__init__()
    spins=0
    while end==False:
        outcome=roulette.play_roulette()
        end,result,trials=gambler.update_money(outcome)
        spins+=1
    episode_wins+=result
    episode_spins[i]=spins
    episode_betswon[i]=gambler.bets_won

print('The probability of winning from simulation of {} episodes is {} in {} days, if daily return is %1'.format(episodes,episode_wins/episodes,trials))
# print('The average number of roulette spins per episode is: ', np.mean(episode_spins))

# Reference : http://web.mit.edu/neboat/Public/6.042/randomwalks.pdf
# Using recursive relation we can obtain true probability of achieving the goal amount using the formula shown below
p=roulette.win_prob

T=gambler.goal_money
n=gambler.initial_money
true_probability=(((1-p)/p)**n-1)/(((1-p)/p)**T-1)
print("The true probability of winning is {}".format(true_probability))



Completed 1000 episodes
Completed 2000 episodes
Completed 3000 episodes
Completed 4000 episodes
Completed 5000 episodes
Completed 6000 episodes
Completed 7000 episodes
Completed 8000 episodes
Completed 9000 episodes
The probability of winning from simulation of 10000 episodes is 1.0 in 168 days, if daily return is %1
The true probability of winning is 1.0
